# EODHD MCP Server - Getting Started

This notebook demonstrates how to use the EODHD MCP Server to fetch financial data.

## Prerequisites

1. Get your API key from [EODHD](https://eodhd.com/)
2. Set the `EODHD_API_KEY` environment variable
3. Install dependencies: `pip install httpx pandas`

In [ ]:
import os
import json
import httpx
import pandas as pd
from datetime import datetime, timedelta

# Configuration
API_KEY = os.getenv("EODHD_API_KEY", "demo")
BASE_URL = "https://eodhd.com/api"

print(f"Using API key: {API_KEY[:4]}..." if len(API_KEY) > 4 else "Using demo key")

## 1. Fetching Historical Stock Prices

In [ ]:
def get_historical_prices(ticker: str, from_date: str, to_date: str) -> pd.DataFrame:
    """Fetch historical EOD prices for a ticker."""
    url = f"{BASE_URL}/eod/{ticker}"
    params = {
        "api_token": API_KEY,
        "fmt": "json",
        "from": from_date,
        "to": to_date
    }
    
    response = httpx.get(url, params=params)
    response.raise_for_status()
    data = response.json()
    
    df = pd.DataFrame(data)
    df['date'] = pd.to_datetime(df['date'])
    df.set_index('date', inplace=True)
    return df

# Fetch AAPL prices for last 30 days
end_date = datetime.now().strftime("%Y-%m-%d")
start_date = (datetime.now() - timedelta(days=30)).strftime("%Y-%m-%d")

aapl_prices = get_historical_prices("AAPL.US", start_date, end_date)
print(f"Fetched {len(aapl_prices)} days of AAPL data")
aapl_prices.tail()

## 2. Getting Live Price Data

In [ ]:
def get_live_price(ticker: str) -> dict:
    """Fetch real-time/delayed price for a ticker."""
    url = f"{BASE_URL}/real-time/{ticker}"
    params = {
        "api_token": API_KEY,
        "fmt": "json"
    }
    
    response = httpx.get(url, params=params)
    response.raise_for_status()
    return response.json()

# Get live price for multiple tickers
tickers = ["AAPL.US", "MSFT.US", "GOOGL.US"]
for ticker in tickers:
    price_data = get_live_price(ticker)
    print(f"{ticker}: ${price_data.get('close', 'N/A')} (Volume: {price_data.get('volume', 'N/A')})")

## 3. Fetching Company Fundamentals

In [ ]:
def get_fundamentals(ticker: str) -> dict:
    """Fetch fundamental data for a company."""
    url = f"{BASE_URL}/fundamentals/{ticker}"
    params = {
        "api_token": API_KEY
    }
    
    response = httpx.get(url, params=params)
    response.raise_for_status()
    return response.json()

# Get AAPL fundamentals
fundamentals = get_fundamentals("AAPL.US")

# Display key metrics
general = fundamentals.get("General", {})
highlights = fundamentals.get("Highlights", {})

print(f"Company: {general.get('Name')}")
print(f"Sector: {general.get('Sector')}")
print(f"Industry: {general.get('Industry')}")
print(f"Market Cap: ${highlights.get('MarketCapitalization', 0):,.0f}")
print(f"P/E Ratio: {highlights.get('PERatio', 'N/A')}")
print(f"EPS: ${highlights.get('EarningsShare', 'N/A')}")

## 4. Technical Indicators

In [ ]:
def get_technical_indicator(ticker: str, function: str, period: int = 14) -> pd.DataFrame:
    """Fetch technical indicator data."""
    url = f"{BASE_URL}/technical/{ticker}"
    params = {
        "api_token": API_KEY,
        "fmt": "json",
        "function": function,
        "period": period
    }
    
    response = httpx.get(url, params=params)
    response.raise_for_status()
    data = response.json()
    
    df = pd.DataFrame(data)
    df['date'] = pd.to_datetime(df['date'])
    df.set_index('date', inplace=True)
    return df

# Get RSI for AAPL
rsi_data = get_technical_indicator("AAPL.US", "rsi", 14)
print("RSI (14) for AAPL:")
rsi_data.tail(10)

## 5. Stock Screener

In [ ]:
def screen_stocks(filters: list, limit: int = 10) -> pd.DataFrame:
    """Screen stocks based on criteria."""
    url = f"{BASE_URL}/screener"
    params = {
        "api_token": API_KEY,
        "filters": json.dumps(filters),
        "limit": limit,
        "sort": "market_capitalization.desc"
    }
    
    response = httpx.get(url, params=params)
    response.raise_for_status()
    return pd.DataFrame(response.json())

# Screen for large-cap tech stocks
filters = [
    ["market_capitalization", ">", 100000000000],  # > $100B
    ["sector", "=", "Technology"]
]

tech_giants = screen_stocks(filters, limit=10)
print("Large-cap Technology Companies:")
tech_giants[["code", "name", "market_capitalization"]].head(10)

## 6. Financial News

In [ ]:
def get_news(ticker: str = None, limit: int = 5) -> list:
    """Fetch financial news."""
    url = f"{BASE_URL}/news"
    params = {
        "api_token": API_KEY,
        "limit": limit
    }
    if ticker:
        params["s"] = ticker
    
    response = httpx.get(url, params=params)
    response.raise_for_status()
    return response.json()

# Get news for AAPL
news = get_news("AAPL.US", limit=5)
print("Latest AAPL News:")
for article in news:
    print(f"- {article.get('title', 'No title')}")
    print(f"  Date: {article.get('date', 'N/A')}")
    print()

## Summary

This notebook demonstrated the basic usage of EODHD API:

1. **Historical Prices** - EOD data for any ticker
2. **Live Prices** - Real-time/delayed quotes
3. **Fundamentals** - Company financials and metrics
4. **Technical Indicators** - RSI, SMA, EMA, MACD, etc.
5. **Stock Screener** - Filter stocks by criteria
6. **Financial News** - Latest market news

For more information, visit:
- [EODHD Documentation](https://eodhd.com/financial-apis/)
- [EODHD MCP Server GitHub](https://github.com/EodHistoricalData/EODHD-MCP-Server)